### Connected Components

In [1]:
import os
from itertools import product

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cufflinks as cf
import plotly.offline
from tqdm import tqdm

from IPython.display import display, HTML
from ipywidgets import interactive_output, HBox, VBox, Layout 
from ipywidgets import Dropdown, Checkbox, ToggleButtons, SelectionRangeSlider, SelectionSlider
from ipywidgets import SelectMultiple
import ipywidgets as widgets

cf.go_offline()
pd.set_option('display.max_columns', 100)

in_dir = os.path.join('output', 'analysis')
dataset = 'youtube'
test_type = 'full'
eval_set = 'test'

result_dir = os.path.join('..', '..', in_dir, dataset, 'eval_{}'.format(eval_set), 'test_{}'.format(test_type))
df = pd.read_csv(os.path.join(result_dir, 'results.csv'))
df.sort_values('num_nodes')

,num_nodes,num_msg_nodes,num_hub_nodes,num_spam_msg_nodes,num_relations,num_edges
18172,3,2,1,0,1,2
14412,3,2,1,0,1,2
14411,3,2,1,0,1,2
8174,3,2,1,0,1,2
14410,3,2,1,0,1,2
...,...,...,...,...,...,...
346,215,159,56,0,2,215
4275,243,240,3,240,2,480
1575,530,527,3,527,2,1054
45,783,569,214,114,2,789


In [12]:
def process_results(df):
    num_components = len(df)
    num_edges = df['num_edges'].sum()

    n_spam_msgs_list = []
    component_size_avg_list = []
    spam_fraction_avg_list = []
    n_components_list = []
    components_sum_list = []

    for n_spam, gf in df.groupby('num_spam_msg_nodes'):

        if n_spam < 1:
            continue

        if n_spam == 0:
            spam_fraction_avg = 0
        else:
            spam_fraction_avg = (gf['num_spam_msg_nodes'] / gf['num_nodes']).mean()

        component_size_avg = gf['num_nodes'].mean()
        components_sum = gf['num_nodes'].sum()

        n_spam_msgs_list.append(n_spam)
        component_size_avg_list.append(component_size_avg)
        spam_fraction_avg_list.append(spam_fraction_avg)
        n_components_list.append(len(gf))
        components_sum_list.append(components_sum)
    
    results = {}
    results['n_spam'] = n_spam_msgs_list
    results['component_size'] = component_size_avg_list
    results['spam_fraction'] = spam_fraction_avg_list
    results['n_components'] = n_components_list
    results['components_sum'] = components_sum_list

    return results

# experiment options
dataset = Dropdown(description='Dataset', options=['youtube', 'twitter', 'soundcloud'], value='youtube')
test_type = ToggleButtons(description='Test type', options=['inductive', 'full'], value='full')
eval_set = ToggleButtons(description='Eval set', options=['val', 'test'], value='test')
# spam_count = SelectionRangeSlider(description='# Spam', options=[0, 1, 1000, 10000000], index=(1, 2))

# create ui
box_1 = VBox([dataset])
box_2 = VBox([eval_set, test_type])
box_3 = VBox([])
ui = HBox([box_1, box_2, box_3])

# # plot graphs
def f(dataset, test_type, eval_set):
    result_dir = os.path.join('..', '..', in_dir, dataset,
                              'eval_{}'.format(eval_set), 'test_{}'.format(test_type))
    fp = os.path.join(result_dir, 'results.csv')

    if not os.path.exists(fp):
        plt.clf()
    
    else:
        df = pd.read_csv(os.path.join(result_dir, 'results.csv'))
    
        if len(df) == 0:
            plt.clf()

        else:

            results = process_results(df)

            fig, axs = plt.subplots(1, 2, figsize=(15, 5))
            axs = axs.flatten()
            
            ax = axs[0]
            ax.plot(results['n_spam'], results['spam_fraction'], label='spam fraction', color='red')
            ax.set_xlabel('# spam in component')
            ax.set_xscale('log')
            ax.set_ylabel('spam fraction')
            ax.legend(loc=4)

            ax1 = ax.twinx()
            ax1.plot(results['n_spam'], results['component_size'], label='component size', color='green')
            ax1.set_ylabel('component size')
            ax1.legend(loc=0)
            
            ax = axs[1]
            ax.bar(results['n_spam'], results['n_components'], label='# components', color='magenta')
            ax.set_xlabel('# spam in component')
            ax.set_xscale('log')
            ax.set_ylabel('# components')
            ax.legend(loc=7)
            
            ax1 = ax.twinx()
            ax1.plot(results['n_spam'], results['components_sum'], label='# msgs in all components', color='orange')
            ax1.set_ylabel('# msgs in all components')
            ax1.legend(loc=0)
            
            plt.tight_layout()
            plt.show()

out = interactive_output(f, {'dataset': dataset, 'test_type': test_type, 'eval_set': eval_set})
display(ui, out)

Output()